<a href="https://colab.research.google.com/github/prabhudc/DQ/blob/master/Credit_Risk_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Risk Analysis

In this project we analysze and attempt at predicting the likelihood of a borrower being able to pay off a loan on time

In [61]:
# The usual suspects
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Modelling
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier


In [2]:
loans_2007 = pd.read_csv('/content/drive/My Drive/DataScience/files/loans_2007.csv')
loans_2007.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## Data Preperation

In [3]:
print(loans_2007.columns)
print('Number of columns : ', len(loans_2007.columns))

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')
Number of columns :  52


Drop features that are not pertinent

In [4]:
drop_columns = ['id','member_id','funded_amnt', 'funded_amnt_inv','grade','sub_grade','emp_title','issue_d', 'zip_code', 'out_prncp', 'out_prncp_inv',
                'total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt']
loans_2007.drop(columns=drop_columns, inplace=True)

We shall consider loan_status to be the target column that conveys if the loan had peen paid on-time

In [5]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

We now format the target variable

* Drop records which do not have the status "Fully Paid" or "Charge Off" 
* Code them to 1 and 0 so as to feed a binary classifier

In [6]:
loans_2007 = loans_2007[loans_2007['loan_status'].isin(['Fully Paid','Charged Off'])].copy()
loans_2007.reset_index(drop=True)
loans_2007.sample(5)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
2724,19425.0,60 months,20.89%,524.32,10+ years,MORTGAGE,46500.0,Verified,Charged Off,n,debt_consolidation,Debit Consolidation,TX,21.19,0.0,Apr-1982,0.0,14.0,0.0,16561.0,70.2%,29.0,f,Mar-2015,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
20888,4000.0,60 months,9.63%,84.27,3 years,MORTGAGE,36000.0,Not Verified,Fully Paid,n,wedding,personal loan,NC,2.43,0.0,Oct-2003,0.0,13.0,0.0,2407.0,34.4%,24.0,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
10495,14000.0,36 months,7.49%,435.43,8 years,MORTGAGE,200000.0,Verified,Fully Paid,n,major_purchase,Trade-up in diamond ring,SC,1.21,0.0,Jan-1999,0.0,5.0,0.0,0.0,0%,19.0,f,Dec-2011,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
10249,28000.0,36 months,7.49%,870.85,< 1 year,MORTGAGE,110000.0,Verified,Fully Paid,n,other,Parents,MD,0.44,0.0,Nov-1985,0.0,10.0,0.0,2029.0,4.3%,35.0,f,Aug-2012,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
19113,6000.0,36 months,5.42%,180.96,3 years,RENT,72000.0,Source Verified,Fully Paid,n,wedding,Wedding Reception Loan,NY,2.15,0.0,Oct-2000,0.0,5.0,0.0,779.0,14.8%,15.0,f,Mar-2014,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [7]:
loans_2007['loan_status'] = loans_2007['loan_status'].apply(lambda x : 1 if x == 'Fully Paid' else 0 )

Drop columns that have just one non-null unique value

In [8]:
columns = loans_2007.columns
drop_columns = []
for col in columns:
  unique_count = loans_2007[col].dropna().unique().shape[0]
  if unique_count == 1:
    drop_columns.append(col)
drop_columns

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [9]:
loans_2007.drop(columns=drop_columns, inplace=True)

In [10]:
loans = loans_2007

Let us check for nulls in this restricted dataset 

In [11]:
null_counts = loans.isnull().sum()
print(null_counts[null_counts>0])
print(loans['pub_rec_bankruptcies'].var())

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64
0.04186729442744287


* employment length can be a strong signal, so we retain it
* pub_rec_bankruticies has a very small variance and hence we wrill drop that as well
* For the columns besides  pub_rec_bankruticies, we are dropping the rows

In [12]:
loans.drop(columns=['pub_rec_bankruptcies'],inplace=True)
loans.dropna(inplace=True)

Now let's focus on the data types and see if they can be consumed by the modelling libraries

In [13]:
loans.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In [14]:
object_columns_df = loans.select_dtypes('object')
object_columns_df.head(1)

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016


In [15]:
# Drop earliest_cr_line and last_credit_pull_d due to complexity in feature engineering
loans.drop(columns=['earliest_cr_line','last_credit_pull_d'], inplace=True)

Explore the remaining textual columns

In [16]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
  print(loans[col].value_counts().head(10))
  print("************************************************************")

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
************************************************************
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
************************************************************
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
Name: emp_length, dtype: int64
************************************************************
 36 months    28234
 60 months     9441
Name: term, dtype: int64
************************************************************
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
Name: addr_state, dtype: int64
************************************************************


In [17]:
cols = ['purpose', 'title']
for col in cols:
  print(loans[col].value_counts().head(10))
  print("************************************************************")

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
Name: purpose, dtype: int64
************************************************************
Debt Consolidation           2068
Debt Consolidation Loan      1599
Personal Loan                 624
Consolidation                 488
debt consolidation            466
Credit Card Consolidation     345
Home Improvement              336
Debt consolidation            314
Small Business Loan           298
Credit Card Loan              294
Name: title, dtype: int64
************************************************************


Remove additional fields that does not reveal information or is cumbersome to use

In [18]:
cols = [ 'addr_state', 'title']
loans.drop(columns=cols,inplace=True)

In [19]:
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype(float)
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype(float)
mapping_dict = {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }


loans['emp_length'] = loans['emp_length'].map(mapping_dict)



Encode categorical variables with dummy variables

In [20]:
# Create dummies
categorial_cols = ["home_ownership","term", "verification_status","purpose"]
dummy_df = pd.get_dummies(loans[categorial_cols])
loans = pd.concat([loans,dummy_df],axis = 1)

# Drop the columns for which dummy variables where introduces
loans.drop(columns=categorial_cols, inplace=True)


## Modelling

In [31]:
# Exclude loan_status for the features
features = loans[loans.columns[~loans.columns.isin(['loan_status'])]]
target = loans['loan_status']

### Logistic Regression

In [59]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
lr.fit(features,target )
# Predict with cross validation
predictions = cross_val_predict(lr,features,target,cv=3)
# Convert to pd.Series
predictions = pd.Series(predictions)

In [60]:
# TPR = TP / (TP + FN)
tp = ((predictions == 1) & (target == 1 )).sum()
fn = ((predictions == 0) & (target == 1 )).sum()
tpr = tp / (tp + fn)
print("TPR : ", tpr)

# FPR = FP / (FP + TN)
fp = ((predictions == 1) & (target == 0 )).sum()
tn = ((predictions == 0) & (target == 0 )).sum()
fpr = fp / (fp + tn)
print("FPR : ", fpr)

TPR :  0.170523072889355
FPR :  0.16722606120434352


### Random Forest

In [63]:
penalty = {
    0: 10,
    1: 1
}

lr = RandomForestClassifier(random_state=1,class_weight=penalty)
lr.fit(features,target )
# Predict with cross validation
predictions = cross_val_predict(lr,features,target,cv=3)
# Convert to pd.Series
predictions = pd.Series(predictions)

# TPR = TP / (TP + FN)
tp = ((predictions == 1) & (target == 1 )).sum()
fn = ((predictions == 0) & (target == 1 )).sum()
tpr = tp / (tp + fn)
print("TPR : ", tpr)

# FPR = FP / (FP + TN)
fp = ((predictions == 1) & (target == 0 )).sum()
tn = ((predictions == 0) & (target == 0 )).sum()
fpr = fp / (fp + tn)
print("FPR : ", fpr)

TPR :  0.9971814368117462
FPR :  0.9970384995064165
